In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = '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',
    model_name = "llama-3.1-70b-versatile"
)
response = llm.invoke("Current prime minister of India")
print(response.content)

The current Prime Minister of India is Narendra Modi. He has been serving as the Prime Minister of India since May 26, 2014.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2819387/software-dev-engineer-game-based-learning")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Dev Engineer, Game-Based Learning - Job ID: 2819387 | Amazon.jobs
Skip to main contentHomeYour job applicationAmazon culture & benefitsDiversity at AmazonLocationsTeamsJob categoriesResourcesInterview tipsDisability accommodationsAbout AmazonFAQ×Software Dev Engineer, Game-Based LearningJob ID: 2819387 | ADSIPL - DelhiApply nowDESCRIPTIONAWS Game-Based Learning Products is looking for a Software Development Engineer to join a fast-paced, startup-minded innovation team where fun is an integral part of your job. Joining our team, you will collaborate with a team of architects, artists, designers, engineers, and testers responsible for the conception, development and operations of highly technical game-based learning solutions that rely on extensive use of hands-on labs, virtual environments, and interactive simulations to get customers to develop practical skills using AWS services.As a Software Development Engineer, you will partner with fellow back-end engineers to build cloud

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Dev Engineer, Game-Based Learning',
 'experience': '3+ years of non-internship professional software development experience, 2+ years of non-internship design or architecture experience',
 'skills': ['Java',
  'C++',
  'C#',
  'Object-oriented design',
  'Full software development life cycle',
  'Games Industry experience',
  'English and Portuguese language proficiency'],
 'description': "Collaborate with a multi-disciplinary team to design, deliver, maintain, and operate multiple game-based learning products. Drive system architecture and promote best practices to produce high quality games. Partner with technical and business stakeholders to influence long-term strategy and establish the team's roadmap. Make data-driven decisions by leveraging industry landscape from engineering, gaming and training, as well as user feedback and telemetry data. Write high quality code."}

In [6]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [7]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
job = json_res
job['skills']

['Java',
 'C++',
 'C#',
 'Object-oriented design',
 'Full software development life cycle',
 'Games Industry experience',
 'English and Portuguese language proficiency']

In [10]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/java-portfolio'}]]

In [11]:
job

{'role': 'Software Dev Engineer, Game-Based Learning',
 'experience': '3+ years of non-internship professional software development experience, 2+ years of non-internship design or architecture experience',
 'skills': ['Java',
  'C++',
  'C#',
  'Object-oriented design',
  'Full software development life cycle',
  'Games Industry experience',
  'English and Portuguese language proficiency'],
 'description': "Collaborate with a multi-disciplinary team to design, deliver, maintain, and operate multiple game-based learning products. Drive system architecture and promote best practices to produce high quality games. Partner with technical and business stakeholders to influence long-term strategy and establish the team's roadmap. Make data-driven decisions by leveraging industry landscape from engineering, gaming and training, as well as user feedback and telemetry data. Write high quality code."}

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Services for Game-Based Learning Solutions

Dear Hiring Manager,

I came across the job description for a Software Dev Engineer, Game-Based Learning, and I was impressed by the innovative approach your company is taking to revolutionize the learning experience. As a Business Development Executive at AtliQ, I'd like to introduce our company and highlight how our expertise can help you achieve your goals.

At AtliQ, we specialize in providing AI and software consulting services that enable businesses to streamline their processes, enhance scalability, and reduce costs. Our team of experts has a proven track record of delivering high-quality solutions that meet the unique needs of our clients.

In particular, our expertise in Java, C++, and C# aligns perfectly with the requirements of your job description. We have a strong portfolio of projects that demonstrate our capabilities in these areas, including:

* Java-based solutions: https://example.com/jav